In [129]:
import pandas as pd
import numpy as np
import os
import pickle
import shutil
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pyreadr
%matplotlib inline

In [130]:
data_folder = '/anvil/projects/tdm/corporate/3rivers-membership/data/'
lending = pyreadr.read_r(data_folder + '/Lending_Table.rds')[None]
print(lending.shape)
    
with open('../data/useful_customers_list.pkl', 'rb') as f:
    customer_list = pickle.load(f)
    
lending = lending[ lending['customerkey'].isin(customer_list) ]
print(lending.shape)
lending.reset_index(inplace=True, drop=True)
lending.head(3)

(260480, 40)
(221167, 40)


,accountnumber,category,chargeoffdate,chargeoffamount,collateraltypecode,commercialrealestatecode,cracode,currentamortizationperiod,currentficoscore,currentmaturitydate,...,productid,productname,productshorttype,producttype,producttypecode,propertytype,remainingterm,timespastdue_30_60,timespastdue_60_90,timespastdue_90_plus
0,127828003.0,CNS,None,0.0,NaN,NaN,OUC,0,NaN,2049-01-31,...,CNS~LOC,Line of Credit,LN,Standard Loan,LOC,Other,305,0,0,0
1,127847003.0,CNS,None,0.0,NaN,NaN,OUC,0,NaN,2049-01-31,...,CNS~LOC,Line of Credit,LN,Standard Loan,LOC,Other,305,0,0,0
2,128213001.0,CNS,None,0.0,NaN,NaN,OUC,37,NaN,2006-04-15,...,CNS~TERM,Term,LN,Standard Loan,TERM,Other,0,0,0,0


Check if customerkey and accountnumber are unique i.e. have one-to-one relation

In [131]:
lending[['customerkey', 'accountnumber']].duplicated().any()

False

customerkey and accountnumber together will serve as primary key.

Overall, we are going to have 3 types of features, binary valued features, features that have no fix scaled i.e. those that need to be standardized, and those that have a fixed range. So, we will have to featurize and preprocess them accordingly.

In [132]:
binary_features, standardization_features = [], []

##### 1. Featurzing feature 'category'

In [133]:
lending_category_dict = lending.category.value_counts().to_dict()
lending_category_dict

{'CNS': 196696, 'MTG': 22742, 'MLN': 909, 'CML': 820}

The total count for categories 'MLN' and 'CML' is less than 1% (around .4% to .5%) which is very small number for the ml model to learn from as compared to other categories. So, we will just mark all categories with 'CNS' as 0 and any other category as 1, making it a binary feature.

In [134]:
lending.category = lending.category.apply(lambda x: 0 if x == 'CNS' else 1)
lending.category.value_counts()

0    196696
1     24471
Name: category, dtype: int64

In [135]:
binary_features.append( 'category' )

##### 2. Featurizing feature chargeoffdate

In [136]:
round((lending.chargeoffdate.isna().sum()/lending.shape[0])*100, 2)

94.87

94.87% of the data is missing for chargeoffdate column. So, we cannot use any imputatation method to impute the missing records but we can take into account the information that chargeoffdate is present for 5.13% by marking these rows with 1 and the rest with 0

In [137]:
lending.chargeoffdate.fillna('', inplace=True)
lending['chargeoffdate_binary'] =  lending.chargeoffdate.apply( lambda x: 0 if x == '' else 1)

In [138]:
binary_features.append( 'chargeoffdate_binary' )

##### 3. chargeoffamount

In [139]:
lending[ lending['chargeoffamount'] == 0].shape[0]/lending.shape[0]

0.95607391699485

In [140]:
np.percentile(lending.chargeoffamount, 95)

0.0

The charge off amount for 95% of the data is 0. Also, this feature makes sense for only those rows/records where the loan amount was actually charged off. As it was charged off for only 5% of the data, charge off amount correpsonding to only those 5% are present (which is obvious). So, going ahead, during feature selection, we might drop this feature as the information that this column holds can be interpreted to some extend by our chargeoffdate_binary feature created earlier. 

##### 4. collateraltypecode 

In [141]:
lending.collateraltypecode.value_counts()

AUTU    103671
SING     39745
TRUK      7293
MCYC      4088
RV        2455
LEAS      2324
OIRE      2046
BOAT      1892
OTNR      1878
OTHR      1419
COND       197
MOBL       177
CD         168
SAV        166
LNDA       137
RENO        46
BASS        43
SNOW        41
LAND        41
SPHM        33
TRAL        24
MOTO        24
EQUP        23
MULT        22
STCK        22
FRMA        18
PUD         15
REOO        11
ATV         11
RSBU        10
HORS         9
PRP1         8
AR           7
CAMP         6
LAAG         5
PRP4         4
ASGN         4
INSP         3
INDU         3
NURS         3
FLOR         3
CHRC         3
PERS         2
BOND         2
PRP2         2
JSKI         2
APT          2
PRP3         2
OFFC         2
HLCR         1
WHSE         1
MOTL         1
FACT         1
Name: collateraltypecode, dtype: int64

In [142]:
lending.shape[0]*.01

2211.67

Except the first couple of values above, all the other values are underrepresented. So, we will just group all the values with count less than 2211 (1% of total records) as 'OTHER' category. 

In [143]:
collateraltype_dict = {'AUTU':103671 ,'SING':39745 ,'TRUK':7293 ,'MCYC':4088,'RV':2455,'LEAS':2324, 'OTHER' : 2211 }
def lending_collateraltypecode_fun(x):
    if x not in ['AUTU','SING','TRUK','MCYC','RV','LEAS']  :
        x = 'OTHER' 
    return collateraltype_dict[x]
lending.collateraltypecode = lending.collateraltypecode.apply(lending_collateraltypecode_fun)
standardization_features.append( 'collateraltypecode' )

##### 5. commercialrealestatecode

In [144]:
round((lending.commercialrealestatecode.isna().sum()/lending.shape[0])*100, 3)

80.352

80% of the values are null values. So, we will create a binary feature that marks 0 if no commercial real estate code is present and 1 otherwise. 

In [145]:
lending.commercialrealestatecode.fillna(0, inplace=True)
lending['commercialrealestatecode_bin'] = lending.commercialrealestatecode.apply(lambda x: 1 if x != 0 else 0)
binary_features.append( 'commercialrealestatecode_bin' )

##### 6. cracode

In [146]:
lending.cracode.isna().sum()
lending.dropna(subset=['cracode'], inplace=True)
lending.cracode.isna().sum()

0

As the number of rows with null values is very small(<1%), we simply drop it. 

In [147]:
lending_cracode_dict = lending.cracode.value_counts().to_dict()
for key, value in lending_cracode_dict.items():
    if value <= 2000 :
        lending_cracode_dict[key] = 1/2000
    else : 
        lending_cracode_dict[key] = 1/value
lending.cracode = lending.cracode.apply(lambda x: lending_cracode_dict[x])
standardization_features.append( 'cracode' )

##### 7. currentAmortizationPeriod

In [148]:
lending.currentamortizationperiod.isna().sum()/lending.shape[0]

0.03656657915846277

Some of the records have null values for amortization period i.e. around 3% of the records. It might be the case that these customers don't have any loan or they might have immediately paid off the loan and hence no amortization period value is available for these records. So, we will just impute the missing values with 0. 

In [149]:
lending.currentamortizationperiod.fillna(0, inplace=True)
standardization_features.append( 'currentamortizationperiod' )

##### 8. currentficoscore

Around 24% of the data is missing for currentficoscore. It follows almost a normal distribution and is slightly left skewed. So it is safe to impute the missing values with median.

In [150]:
lending.currentficoscore.fillna(lending.currentficoscore.median(), inplace=True)
standardization_features.append( 'currentficoscore' )

##### 9. currentmaturitydate

In [151]:
print(lending.currentmaturitydate.isna().sum())
lending.dropna(subset=['currentmaturitydate'], inplace=True)
lending['currentmaturitydate'] = pd.to_datetime(lending.currentmaturitydate)
lending['currentmaturity_month'] = lending.currentmaturitydate.dt.month
lending['currentmaturity_day'] = lending.currentmaturitydate.dt.day
lending['currentmaturity_year'] = lending.currentmaturitydate.dt.year
standardization_features.extend([ 'currentmaturity_month', 'currentmaturity_day', 'currentmaturity_year' ])

8


##### 10. currentprincipalbalance

In [152]:
standardization_features.append('currentprincipalbalance')

##### 11. dayspastdue

In [153]:
standardization_features.append('dayspastdue')

##### 12. haslineofcredit

It is a binary feature, so we will just label the entries with '0' and '1' for 'No' and 'Yes' labels respectively.

In [154]:
lending['haslineofcredit_bin'] = lending.haslineofcredit.apply(lambda x: 0 if x == 'No' else 1)
binary_features.append('haslineofcredit_bin')

##### 13.loantype

There are 4 different types of loans, the majority being 'consumer loan' followed by 'mortgage loan'.'Master line' and 'Commercial loan' have a count of less than 1000 and therefore are seriously underrepresented. So, we can convert this feature into a binary feature, with 'consumer loan' labelled as 0 and the rest as 1.

In [155]:
lending['loantype_bin'] = lending.loantype.apply(lambda x : 0 if x == 'Consumer Loan' else 1)
binary_features.append( 'loantype_bin' )

In [156]:
lending.loantype_bin.value_counts()

0    195842
1     23558
Name: loantype_bin, dtype: int64

##### 14. hasrevolvinglineofcredit

It is a binary feature, so we will just label the entries with '0' and '1' for 'No' and 'Yes' labels respectively.

In [157]:
lending['hasrevolvinglineofcredit_bin'] = lending.hasrevolvinglineofcredit.apply(lambda x : 0 if x == 'No' else 1)
sum(lending['hasrevolvinglineofcredit_bin'] == lending['haslineofcredit_bin'])/lending['haslineofcredit_bin'].shape[0]

0.9999589790337283

As hasrevolvinglineofcredit_bin and haslineofcredit_bin are conveying almost the same information, we will just use either of them.

##### 15. Interestrate

In [158]:
standardization_features.append('interestrate')

##### 16. interestratevariable

In [159]:
lending.interestratevariable.isna().sum()/lending.shape[0]

0.21297629899726528

21% of the values are missing. The data dictionary says that interestratevariable contains 3 values 'floating', 'variable' and 'ARM' (Adjustable Rate Mortgage), whereas in reality the values found are 'fix' and 'var' (variable) with 21% missing values. 

##### 16. ischargedoff

ischargedoff is a binary values feature with 95% of them values as 'no' and the rest as 'yes'. So, we will just mark them as 0 for no and 1 for yes.

In [160]:
lending['ischargedoff_bin'] = lending.ischargedoff.apply(lambda x: 0 if x == 'No' else 1)
binary_features.append('ischargedoff_bin')

##### 17. iscollateralprimary

Here, for around 75% of the data for this feature has value 'Yes' and for the rest it is none. So, it can be assumed that for those rows with none value, the collateral is not primary. So, we will impute all the null values with 0 and substitute 1 for rows that say 'Yes' for this feature, thus making it a binary feature.

In [161]:
lending.iscollateralprimary.fillna(0, inplace=True)
lending[ 'iscollateralprimary_bin'] = lending.iscollateralprimary.apply(lambda x: 0 if x == 0 else 1)
binary_features.append( 'iscollateralprimary_bin' )

##### 18. isinterestratevariable

The features 'isinterestratevariable' and 'interestratevariable' seem to contain the same information, the only different being one is filled with binary values as 'Yes' and 'No' whereas the other variables represents the same information using 'FIX' and 'VAR'. Thus, 'isinterestratevariable' seems to be an indicator variable for interestratevariable, as found in EDA.

In [162]:
lending[ 'isinterestratevariable_bin' ] = lending.isinterestratevariable.apply(lambda x : 1 if x == 'Yes' else 0 )
binary_features.append( 'isinterestratevariable_bin' )

##### 19. monthlypayment

In [163]:
standardization_features.append( 'monthlypayment')

##### 20. monthspastdue

In [164]:
standardization_features.append( 'monthspastdue' )

##### 21. originalficodate

originalficodate is missing for 30% of the data. The traditional means of imputing missing value like mean, median and mode won't really be helpful so we might not include this feature. Or one way to use this feature is to create a binary valued feature, where we given '1' if the value is present and '0' for an absent/none value. So, we will just convert it into binary valued feature in this fashion.

In [165]:
lending.originalficodate.fillna( 0,inplace=True)
lending['originalficodate_bin'] = lending.originalficodate.apply( lambda x : 0 if x==0 else 1)
binary_features.append( 'originalficodate_bin' )

##### 22. originalficoscore

In [166]:
lending.originalficoscore.isna().sum()/lending.shape[0]

0.2034275296262534

In [167]:
median_ficoscore = lending[ lending.originalficoscore != 0 ]['originalficoscore'].median()

Around 3000 rows have 0 ficoscore which is not possible as ficoscore ranges in 350 to 800 range. Those rows with 0 as ficoscore are actually null values. So, we will impute all the missing values with median fico score of those rows where ficoscore wasn't zero. Besides, we will replace all occurences of '0' ficoscore with median ficoscore value.

In [168]:
lending.fillna( median_ficoscore, inplace=True)
lending['originalficoscore'] = lending.originalficoscore.apply( lambda x : median_ficoscore if x == 0 else x)
standardization_features.append( 'originalficoscore' )

##### 23. originalprincipalamount

In [169]:
standardization_features.append( 'originalprincipalamount' )

##### 24. originalterm

There are some records where original term is negative, which is the number of months and therefore cannot be negative. So, these values are outliers, probably due to data entry errors and therefore, we will fill in all the negative values with median values. 

In [170]:
median_originalterm = lending[lending.originalterm > 0 ].originalterm.median()

In [171]:
lending['originalterm'] = lending.originalterm.apply(lambda x: median_originalterm if x <= 0 else x ) 
standardization_features.append( 'originalterm') 

##### 25. originationdate

In [172]:
lending.originationdate = pd.to_datetime(lending.originationdate, format="%Y-%m-%d")
lending[ 'originationdate_day' ] = lending.originationdate.dt.day
lending[ 'originationdate_month' ] = lending.originationdate.dt.month
lending[ 'originationdate_year' ] = lending.originationdate.dt.year
standardization_features.extend( [ 'originationdate_day', 'originationdate_month', 'originationdate_year' ] )

##### 26. paymentfrequency

Based on the EDA, less than 500 records have payment frequency other than 'monthly', which is very less for the ml model to learn any pattern or draw any insights from, as compared to the total data size. So, we will exempt this feature from final set of features.

##### 27. payoffdate

In [173]:
lending.payoffdate.isna().sum()/ lending.payoffdate.shape[0]

0.0

Around 28% of the values for payoffdate are missing this might be because these 28% of customers haven't paid off the loan yet. So, we can convert this feature into binary values feature, turning all missing values into '1' and the rest in '0', where '1' indicates that the loan is yet to be paid off and '0' indicates that the loan has been paid off.

In [174]:
lending.payoffdate.fillna(1, inplace=True)
lending[ 'payoffdate' ] = lending.payoffdate.apply( lambda x : 1 if x == 1 else 0 )
binary_features.append('payoffdate')

##### 28. productid 

With the exception of the first 10 values (when arranged in decreasing order of value counts) all the other values have count of less than 5000 i.e. less than 2%. So, we will club these all under the category of 'OTHER' and further standardize the values based on their value count.

In [175]:
productid_dict = lending.productid.value_counts().to_dict()
for key,value in productid_dict.items():
    if value < 5000: 
        productid_dict[key] = 5000
        
lending[ 'productid' ] = lending.productid.apply( lambda x : productid_dict[x] )
standardization_features.append( 'productid' )

##### 29. productname

productname feature is kind of alias for product id and therefore as we have already used the product id feature, we can drop the productname feature otherwise it will introduce redundancy in data.

##### 30. productshorttype and producttype

Based on EDA, we can say that these two features represent almost the same information. 

In [176]:
producttype_dict = lending.producttype.value_counts().to_dict()
lending[ 'producttype' ] = lending.producttype.apply( lambda x : producttype_dict[x] )
standardization_features.append( 'producttype' )

##### 30. producttypecode

From EDA, we can conclude that producttypecode is almost same as product id and therefore we can skip it or it might lead to correlated features thus introducing data redundancy. 

##### 31. propertytype 

With the exception of the first 10 values (when arranged in descending order of value counts), all values are less than 200, which is very small number for the model to draw any meaningful and useful insights. So, we will just assume the count of all such values as 200 and then standardize the feature.

In [177]:
propertytype_dict = lending.propertytype.value_counts().to_dict()
for key,value in propertytype_dict.items():
    if value < 200 : 
        propertytype_dict[key] = 200
lending[ 'propertytype' ] = lending.propertytype.apply(lambda x : propertytype_dict[x])
standardization_features.append( 'propertytype' )

##### 32. remainingterm, timespastdue_30_60, timespastdue_60_90, timespastdue_90_plus

In [178]:
standardization_features.extend( [ 'remainingterm', 'timespastdue_30_60', 'timespastdue_60_90', 'timespastdue_90_plus'])

##### 33. KLI

In [179]:
kli = pyreadr.read_r(data_folder + '/KeyLifestyleIndicator_Table.rds')[None]
with open('../data/useful_customers_list.pkl', 'rb') as f:
    customers_list = pickle.load(f)
    
kli = kli[ kli.CustomerID.isin(customers_list)]
temp_kli = kli.groupby('CustomerID', group_keys=True).count()
lending = lending.merge( temp_kli, left_on = 'customerkey', right_on = 'CustomerID', how='left')
median_kli = lending[lending.KLI != 0 ].KLI.median()
lending.KLI.fillna(median_kli, inplace=True)
standardization_features.append('KLI')

In [180]:
X_labeled = lending[ ['accountnumber', 'customerkey'] + standardization_features + binary_features]

In [181]:
# X_labeled.to_csv('../data/X_train.csv', index=None)

In [182]:
from sklearn.preprocessing import StandardScaler
X = lending[ standardization_features + binary_features ]
scaler = StandardScaler()
scaler.fit(X)
vectorized_X = scaler.transform(X)

# Save the numpy vector to a file
# with open('vectorized_X.npy', 'wb') as f:
#     np.save(f, vectorized_X)